## 测试分组回测（group_trader）

In [6]:
from billions.data_loader import *
from billions.back_trader import *

name = 'betastd'
data = loader_factor_close(
    'data/' + name + '.csv', 'data/close.csv', 'csv')

# run trade
gt = group_trader(data, number=5, name=name)
gt.run(peride='M')
gt.summary()


## 测试海龟

In [7]:
from billions.data_loader import *
from billions.back_trader import *

class turtle(trader):
    def __init__(self, data, total_amount=1000000, peride='D'):
        super().__init__(data, total_amount=total_amount, peride=peride)
        self.codes = np.unique(np.array(list(data.index))[:, 1])

        self.data = data

    def trade(self, date, prices):
        for code in self.codes:
            today = self.dict_data[(date, code)]

            if today['close'] > today['up']:
                self.profolio.buy(code, 100, prices[code])

            if today['close'] < today['down']:
                if code in self.profolio.profolio.keys():
                    self.profolio.sell(code, self.profolio.profolio[code]['amount'], prices[code])
                # self.profolio.sell(code, today.close*100)


data = load_price('data/600000.csv')
data['up'] = data['high'].shift().rolling(20).apply(np.max, raw=False)
data['down'] = data['low'].shift().rolling(20).apply(np.min, raw=False)
data['pre_close'] = data['close'].shift()
data = data[20:]

t = turtle(data)
t.run()
t.summary()
